**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Apr  7 21:23:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
%cd /gdrive/Shareddrives/'Departamento de Tecnología FONSIARA'/Proyectos/'[P-03] Satellite Image Detection'/sid_ml_analysis

/gdrive/Shareddrives/Departamento de Tecnología FONSIARA/Proyectos/[P-03] Satellite Image Detection/sid_ml_analysis


**1) Clone the Darknet**



In [4]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 14751 (delta 0), reused 3 (delta 0), pack-reused 14748
Receiving objects: 100% (14751/14751), 13.31 MiB | 6.16 MiB/s, done.
Resolving deltas: 100% (10031/10031), done.
Checking out files: 100% (2023/2023), done.


**2) Compile Darknet using Nvidia GPU**


In [5]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/gdrive/Shareddrives/Departamento de Tecnología FONSIARA/Proyectos/[P-03] Satellite Image Detection/sid_ml_analysis/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: thi

**3) Configure Darknet network for training YOLO V3**

In [6]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [7]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [9]:
# Create folder on google drive so that we can save there the weights
!mkdir "/gdrive/Shareddrives/Departamento de Tecnología FONSIARA/Proyectos/[P-03] Satellite Image Detection/sid_ml_analysis/yolov3"

In [10]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /gdrive/Shareddrives/Departamento de Tecnología FONSIARA/Proyectos/[P-03] Satellite Image Detection/sid_ml_analysis/yolov3' > data/obj.data
!mkdir data/obj

In [11]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-04-07 21:29:19--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  55.8MB/s    in 2.8s    

2021-04-07 21:29:22 (55.8 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [21]:
!pwd

/gdrive/Shareddrives/Departamento de Tecnología FONSIARA/Proyectos/[P-03] Satellite Image Detection/sid_ml_analysis/darknet


In [22]:
!unzip "/gdrive/Shareddrives/Departamento de Tecnología FONSIARA/Proyectos/[P-03] Satellite Image Detection/sid_ml_analysis/yolov3/images.zip" -d data/obj

Archive:  /gdrive/Shareddrives/Departamento de Tecnología FONSIARA/Proyectos/[P-03] Satellite Image Detection/sid_ml_analysis/yolov3/images.zip
   creating: data/obj/train/
  inflating: data/obj/__MACOSX/._train  
  inflating: data/obj/train/1rFB4qso6-NTjDSUzLxm42hRz_cpK7woT_4_png.rf.541927754e1026398a4ba01f11e3f331.jpg  
  inflating: data/obj/__MACOSX/train/._1rFB4qso6-NTjDSUzLxm42hRz_cpK7woT_4_png.rf.541927754e1026398a4ba01f11e3f331.jpg  
  inflating: data/obj/train/1vpJ1VxUvT6Uc4PdkX-9v7TE08_1zV36Q_2_png.rf.d77ad45f34f7d086741eeac0170a4030.txt  
  inflating: data/obj/__MACOSX/train/._1vpJ1VxUvT6Uc4PdkX-9v7TE08_1zV36Q_2_png.rf.d77ad45f34f7d086741eeac0170a4030.txt  
  inflating: data/obj/train/70_JPG.rf.9a11f4096bb8355def6953546c2387ea.txt  
  inflating: data/obj/__MACOSX/train/._70_JPG.rf.9a11f4096bb8355def6953546c2387ea.txt  
  inflating: data/obj/train/1vpJ1VxUvT6Uc4PdkX-9v7TE08_1zV36Q_0_png.rf.161d1ba9ed344fd33c865da561936059.txt  
  inflating: data/obj/__MACOSX/train/._1vpJ1VxUvT

In [30]:
!ls data/obj

__MACOSX  train


In [31]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/train/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Streaming output truncated to the last 5000 lines.

['0', '0.1658653846153846', '0.9579326923076923', '0.036057692307692304', '0.04447115384615385']
276 data/obj/train/96_JPG.rf.7123ab61746b8a257aff72412cf77f2e.txt
0 0.1658653846153846 0.9579326923076923 0.036057692307692304 0.04447115384615385
0 0.34134615384615385 0.96875 0.040865384615384616 0.06009615384615385

['0', '0.34134615384615385', '0.96875', '0.040865384615384616', '0.06009615384615385']
276 data/obj/train/96_JPG.rf.7123ab61746b8a257aff72412cf77f2e.txt
0 0.34134615384615385 0.96875 0.040865384615384616 0.06009615384615385
0 0.22956730769230768 0.9747596153846154 0.028846153846153848 0.04447115384615385

['0', '0.22956730769230768', '0.9747596153846154', '0.028846153846153848', '0.04447115384615385']
276 data/obj/train/96_JPG.rf.7123ab61746b8a257aff72412cf77f2e.txt
0 0.22956730769230768 0.9747596153846154 0.028846153846153848 0.04447115384615385
0 0.5685096153846154 0.9747596153846154 0.06850961538461539 0.04807692307692308

In [33]:
import glob
images_list = glob.glob("data/obj/train/*.jpg")
print(images_list)

['data/obj/train/1rFB4qso6-NTjDSUzLxm42hRz_cpK7woT_4_png.rf.541927754e1026398a4ba01f11e3f331.jpg', 'data/obj/train/24_JPG.rf.cedd3dd24f85bcb4baf1f73667e081c7.jpg', 'data/obj/train/1eOflxKBkjZ2J_v-8B_YNoY_J2Jfu0mCj_9_png.rf.786b9cf004960abd6a623ecd9fa2f689.jpg', 'data/obj/train/1HoYJjr09ARcNXW2R6M9fMDNlnR4BERO8_1_png.rf.1bbbfaa30075ba7c7a5d3a19d42c23c3.jpg', 'data/obj/train/1eOflxKBkjZ2J_v-8B_YNoY_J2Jfu0mCj_7_png.rf.a9d335cf18dd0cac6facc2846310954f.jpg', 'data/obj/train/1LUQPPfGY5BKIL5zSyhsIwSimmNRkyvCM_9_png.rf.5376201b8b57bf1248d6814b89eb553d.jpg', 'data/obj/train/1X-yqZQpq2btq_TW4oJCc498aCpXLLfB2_2_png.rf.c1bb08b9f4a58e5a3a68ade44206a0a6.jpg', 'data/obj/train/1Oii_dYEviU6ie2nTX5UEp7t_EzGTGrtI_8_png.rf.74526d210e8b24342f5ac20f7e7e06f7.jpg', 'data/obj/train/1IgGMBI_iOLn9mi8MA6xTqrDtc5b1VPib_2_png.rf.56c2b4346e37f35573a63c863d609103.jpg', 'data/obj/train/1ERUqn0IWwczeIWYqkTJhv61XOVB5IExf_8_png.rf.01de4e95f928d0e168cc9f3587bddc54.jpg', 'data/obj/train/1QiCmz5bVr5t-aI5IW01SDdfOoY_8degr_6_

In [34]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.678245), count: 12, class_loss = 3.128861, iou_loss = 1.292513, total_loss = 4.421374 
 total_bbox = 1595188, rewritten_bbox = 0.068519 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.023756, iou_loss = 0.000000, total_loss = 0.023756 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.768295), count: 31, class_loss = 2.910756, iou_loss = 1.602800, total_loss = 4.513556 
 total_bbox = 1595219, rewritten_bbox = 0.068517 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v

In [1]:
!chmod 755 -R '/gdrive/Shareddrives/Departamento de Tecnología FONSIARA/Proyectos/[P-03] Satellite Image Detection/sid_ml_analysis'

chmod: cannot access '/gdrive/Shareddrives/Departamento de Tecnología FONSIARA/Proyectos/[P-03] Satellite Image Detection/sid_ml_analysis': No such file or directory
